In [1]:
# Using langchain initialize local LLM model

from langchain_community.chat_models import ChatOllama

chat_model = ChatOllama(model="llama2:7b-chat",)



In [2]:
from langchain.schema import HumanMessage

messages = [HumanMessage(content="Tell me about the history of AI")]
chat_model(messages)

/home/c3po/mambaforge/envs/dev-llm/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


AIMessage(content='\nArtificial intelligence (AI) has a rich and diverse history spanning several decades. Here\'s an overview:\n\n1. 1950s-60s: The Dartmouth Conference and the Birth of AI\nThe term "Artificial Intelligence" was coined at a conference held at Dartmouth College in 1956. Attendees included computer scientists, mathematicians, and cognitive scientists who were interested in exploring ways to simulate human intelligence using machines. This marked the beginning of AI as a field of research.\n2. 1951: The Turing Test\nAlan Turing, a British mathematician and computer scientist, proposed a test to measure a machine\'s ability to exhibit intelligent behavior equivalent to, or indistinguishable from, that of a human. The Turing Test has since become a benchmark for measuring the success of AI systems.\n3. 1956: The First AI Program\nJohn McCarthy, a computer scientist, created the first AI program, called the Logical Theorist, which was designed to reason and solve problems u

In [3]:
from langchain.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int


@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent


tools = [multiply, add, exponentiate]

In [ ]:
# from langchain.agents import AgentType
# from langchain.agents import initialize_agent

# agent = initialize_agent(
#     tools=tools,
#     agent_type=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
#     llm=chat_llm,
#     verbose=True
# )

In [5]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

In [6]:
from langchain.agents import AgentExecutor, create_react_agent

agent = create_react_agent(chat_model, tools=tools, prompt=prompt)


In [9]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": " <INST> Only use a tool if needed, otherwise respond with Final Answer.</INST> Calculate 5*3?"})



> Entering new AgentExecutor chain...
Question: What is the result of 5 multiplied by 3?
Thought: I can multiply 5 and 3 using the `multiply` function.
Action: `multiply(5, 3)`
Action Input: 5 and 3`multiply(5, 3)` is not a valid tool, try one of [multiply, add, exponentiate].Question: What is the result of 5 multiplied by 3?
Thought: I can multiply 5 and 3 using the `add` function.
Action: `add(5, 3)`
Action Input: 5 and 3`add(5, 3)` is not a valid tool, try one of [multiply, add, exponentiate].Question: Calculate 5*3.

Thought: I can multiply 5 and 3 using the `exponentiate` function.
Action: `exponentiate(5, 3)`
Action Input: 5 and 3`exponentiate(5, 3)` is not a valid tool, try one of [multiply, add, exponentiate].Question: Calculate 5*3.

Thought: I can multiply 5 and 3 using the `multiply` function.
Action: `multiply(5, 3)`
Action Input: 5 and 3`multiply(5, 3)` is not a valid tool, try one of [multiply, add, exponentiate].Question: Calculate 5*3.

Thought: I can multiply 5 and 3

{'input': ' <INST> Only use a tool if needed, otherwise respond with Final Answer.</INST> Calculate 5*3?',
 'output': '15'}